⚠️ If you are mounting your google drive in Colab, run the following cell.

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install numpy

In [9]:
import os
from collections import Counter
from  tqdm import tqdm
import numpy as np
import pandas as pd

In [10]:
%%writefile install.sh
current=$(pwd)

# download boost
wget https://boostorg.jfrog.io/artifactory/main/release/1.77.0/source/boost_1_77_0.tar.gz
tar -xf boost_1_77_0.tar.gz

# install boost
cd boost_1_77_0

./bootstrap.sh --with-libraries=program_options,iostreams
./b2 install

cd $current

Writing install.sh


In [11]:
!sh install.sh &> /dev/null

In [12]:
%%writefile install-prereq.sh

# install seqtk
current=$(pwd)
git clone https://github.com/lh3/seqtk.git;
cd seqtk; make

# install wtdbg2
cd $current
git clone https://github.com/ruanjue/wtdbg2
cd wtdbg2 && make

Writing install-prereq.sh


In [13]:
!sh install-prereq.sh &> /dev/null

In [17]:
!cp /content/drive/MyDrive/FYP/FYP/test/reads.fasta ./reads.fasta

In [15]:
if not os.path.exists('output'): os.mkdir('output')

In [16]:
exp = "./output/"

In [19]:
!grep ">" ./reads.fasta > $exp/read_ids

In [20]:
%%writefile filter_alignments.py

import sys
import fileinput
import gzip
import numpy as np

path = "./"

if len(sys.argv) == 2:
    path = sys.argv[1] + "/"

class Alignment:
    def __init__(self, line):
        """
        COL1 qry_name
        COL2 qry_strand
        COL3 qry_length
        COL4 qry_beg
        COL5 qry_end
        COL6 ref_name
        COL7 ref_strand (always equals +)
        COL8 ref_length
        COL9 ref_beg
        COL10 ref_end
        COL11 match_len (length of matched k-mers)
        COL12 align_len (length of aligned)
        COL13 #kcnt (number of matched k-mers)
        COL14 #gap (number of gapped BINs)
        COL15 cigar (256 x SAM's cigar)
        """
        data = line.strip().split("\t")
        self.raw_data = line.strip()
        self.qry_name = data[0]
        self.qry_strand = data[1]
        self.qry_length = int(data[2])
        self.qry_beg = int(data[3])
        self.qry_end = int(data[4])
        self.ref_name = data[5]
        self.ref_strand = data[6]
        self.ref_length = int(data[7])
        self.ref_beg = int(data[8])
        self.ref_end = int(data[9])
        self.match_len = int(data[10])
        self.align_len = int(data[11])
        self.kmers = int(data[12])
        self.gap = int(data[13])


def is_overlap(alignment):
    qry_beg = alignment.qry_beg
    qry_end = alignment.qry_end
    ref_beg = alignment.ref_beg
    ref_end = alignment.ref_end
    qry_length = alignment.qry_length
    ref_length = alignment.ref_length

    THRESHOLD = 512

    # full overlap
    if qry_beg <= THRESHOLD and qry_length - qry_end <= THRESHOLD:
        return True
    elif ref_beg <= THRESHOLD and ref_length - ref_end <= THRESHOLD:
        return True

    # qry end overlap
    if qry_length - qry_end <= THRESHOLD and ref_beg <= THRESHOLD:
        return True
    # ref end overlap
    elif ref_length - ref_end <= THRESHOLD and qry_beg <= THRESHOLD:
        return True

    return False

def process_batch(alignments, fpe, fpd):
    # skip alignments that are self, this can cause total failure
    # skip non overlaps
    alignments = [a for a in alignments if is_overlap(a) and a.qry_name!=a.ref_name]
    # exit if empty (first scenario)
    if len(alignments) == 0:
        return

    # compute alignment overlaps
    alignments = sorted(alignments, key=lambda a: a.match_len, reverse=True)
    match_lengths = [a.match_len for a in alignments]
    mean_match = np.mean(match_lengths)

    degree = 0
    for n, a in enumerate(alignments):
        # record actual edge count
        degree += 1

        # write only top 20 edges
        if n < 20:
            fpe.write(f"{a.qry_name}\t{a.ref_name}\n")

    fpd.write(f"{alignments[0].qry_name}\t{degree}\n")

active_query = None
alns_buffer = []
out_file_edges = open(path + 'reads.alns', 'w+')
out_file_degree = open(path + 'degree', 'w+')

for line in fileinput.input('-'):
    if len(line.strip()) == 1:
        continue

    alignment = Alignment(line)

    if alignment.qry_name != active_query:
        # new query
        # if there is a previous query process it
        if len(alns_buffer) > 0:
            process_batch(alns_buffer, out_file_edges, out_file_degree)
            # sys.exit(0)

        # reset buffers
        active_query = alignment.qry_name
        alns_buffer = [alignment]
    else:
        alns_buffer.append(alignment)

if len(alns_buffer) > 0:
    process_batch(alns_buffer, out_file_edges, out_file_degree)

out_file_edges.close()
out_file_degree.close()

Writing filter_alignments.py


In [21]:
!./wtdbg2/kbm2  -i ./reads.fasta -d ./reads.fasta -n 2000 -l 2560 -t 16 | python filter_alignments.py $exp/

--
-- total memory       13290472.0 kB
-- available          11680824.0 kB
-- 2 cores
-- Starting program: ./wtdbg2/kbm2 -i ./reads.fasta -d ./reads.fasta -n 2000 -l 2560 -t 16
-- pid                     52262
-- date         Sat Apr 13 15:11:41 2024
--
[Sat Apr 13 15:11:41 2024] loading sequences
179244 reads
[Sat Apr 13 15:12:06 2024] Done, 179244 reads, 3734858266 bp, 14499904 bins
[Sat Apr 13 15:12:06 2024] indexing, 16 threads
[Sat Apr 13 15:12:06 2024] - scanning kmers (K0P21S4.00) from 14499904 bins
14499904 bins
** PROC_STAT(0) **: real 414.842 sec, user 451.990 sec, sys 24.480 sec, maxrss 5174616.0 kB, maxvsize 6427924.0 kB
[Sat Apr 13 15:18:36 2024] - high frequency kmer depth is set to 1000
[Sat Apr 13 15:18:38 2024] - Total kmers = 205083239
[Sat Apr 13 15:18:38 2024] - average kmer depth = 2
[Sat Apr 13 15:18:38 2024] - 0 low frequency kmers (<1)
[Sat Apr 13 15:18:38 2024] - 527 high frequency kmers (>1000)
[Sat Apr 13 15:18:38 2024] - indexing 205082712 kmers, 593583939 i

In [22]:
exp = "./output/"

alignments_file_path = exp + "reads.alns"
degrees_file_path = exp + "degree"

comp = pd.read_csv(exp + "4-mers.tsv", header=None).to_numpy()

In [23]:
comp.shape

(179244, 136)

In [24]:
!grep -c '>' reads.fasta

179244


In [31]:
def get_idx_maps(read_ids_file_path):
    read_id_idx = {}
    # global read_id_idx
    with open(read_ids_file_path) as read_ids_file:
        for rid in tqdm(read_ids_file):
            rid = rid.strip().split()[0][1:]
            read_id_idx[rid] = len(read_id_idx)

    return read_id_idx


def load_read_degrees(degrees_file_path,read_id_idx):
    degree_array = np.zeros((len(read_id_idx),), dtype=int)
    for line in tqdm(open(degrees_file_path, 'r')):
        i, d = line.strip().split()
        d = int(d)
        # print(degree_array)
        degree_array[read_id_idx[i]] = d

    return degree_array


def load_edges_as_numpy(edges_txt_path, edges_npy_path):
    if not os.path.isfile(edges_npy_path):
        edges_txt = [x.strip() for x in tqdm(open(edges_txt_path))]
        edges = np.zeros((len(edges_txt), 2), dtype=np.int32)

        for i in tqdm(range(len(edges_txt))):
            e1, e2 = edges_txt[i].strip().split()
            edges[i]  = [int(e1), int(e2)]

        np.save(edges_npy_path, edges)

    return np.load(edges_npy_path)

def alignments_to_edges(alignments_file_path, edges_txt_path, read_id_idx):
    TP = 0
    FP = 0

    if not os.path.isfile(edges_txt_path):
        with open(edges_txt_path, "w+") as ef:
            for line in tqdm(open(alignments_file_path, "r")):
                u, v = line.strip().split('\t')

                if u == v:
                    continue
                try:
                    ef.write(f"{read_id_idx[u]}\t{read_id_idx[v]}\n")
                except:
                    print(f'Missing {u,v}')

In [32]:
read_id_idx = get_idx_maps(exp + 'read_ids')
degree_array = load_read_degrees(degrees_file_path,read_id_idx)
alignments_to_edges(exp+"reads.alns", exp + "edges.txt", read_id_idx)

edges = load_edges_as_numpy(exp + "edges.txt", exp + "edges.npy")
sample_weights = np.zeros_like(degree_array, dtype=np.float32)
sample_scale = np.ones_like(degree_array, dtype=np.float32)

179244it [00:00, 322114.30it/s]
177746it [00:00, 409992.40it/s]
3412328it [00:12, 267815.36it/s]
3412328it [00:01, 2023785.84it/s]
100%|██████████| 3412328/3412328 [00:04<00:00, 753745.45it/s]


In [33]:
for n, d in enumerate(degree_array):
    sample_weights[n] = 1.0/d if d>0 else 0
    sample_scale[n] = max(1, np.log10(max(1, d)))

scaled = comp * sample_scale.reshape(-1, 1)

In [34]:
np.savez(exp + 'data.npz', edges=edges, scaled=scaled)